In [1]:
# -*- coding: utf-8 -*-

機械学習で東京23区のお買い得物件を探してみる

# ライブラリーインポート

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import os
import csv
import collections
from collections import Counter
from tqdm import tqdm
import re
import itertools

# データ取得の検討

BeautifulSoupでSuumoよりスクレイピング。

## 足立区のurl取得

## データ（属性）取得

### 情報の取得方法（検討用）

=> これで実施していく。

## データ取得実施

=> 219/222:4 で、サイトを調べたところ、219ページまでしかなかった。<br>
=> urls取得日からデータ取得日が4日後のため情報が更新されていたことが原因のため、エラーは無視して作業を進める。

## データフレーム作成・csv保存

# データ前処理の検討

## csvからdf作成

## 賃料がNAの行を削除

## 列を分割

### 住所

### 立地

- 「◇路線/△駅 歩◯分」「◇路線/△駅 バス□分 歩◯分」の２種類がある。
- => ["路線", "駅", "バス", "徒歩"]:[◇,△,□,◯]　とする。（※バスがない場合は□=0）

↓実際はデータ数が多いため、5,000行ずつぐらいで処理を行う。

### 築年数

=> 「新築」を「築0年」と変換して数値を取得する。

### 建物高さ

- ほとんどの建物に地下がないことがわかる。
- => 部屋の階数変換として、地下はマイナス値とするため、項目「地上」と「地下」にし、地下はマイナス値とする。
- 「平屋」は1階建とする。

### 階

- 地下の部屋は1件しかない。
- => 「-」は1とし、「○-△階」は○階とする。また、地下はマイナス値とする。

### 賃料

- => 全て「〇〇万円」表記なので、〇〇のみ取得

### 管理費

- 「-」を0円とし、数値のみ取得

### 敷/礼/保証/敷引,償却

- "/"で分割し、「-」は0円とし、数値を取得する。
- 万円単位なので10,000倍する。

### 間取り

- Suumoでの部屋検索より、「部屋数」「D有無」「K有無」「L有無」「S有無」に分割する。

### 専有面積

- => 全て"m"を削除し数値化

## 列の作成・結合など

- ここで、参考サイトより、求めたい「賃料」＝「管理費を含む月額費用」と定義すること、「敷/礼/保」＝「初期費用」と捉え、「月額費用」と「初期費用」を追加する。

### 月額費用・初期費用

- また、参考サイトにはなかったが、利用路線数も入れておく。

### 利用路線数

## データフレーム作成・csv保存

- df["レンタルタイプ"], df_address, df_locations, df_age, df_height2, df_floor["階"]
- df_rent["賃料"], df_admin["管理費"], df_others, df_floor_plan, df_area
- df_cost

### 全てのデータフレームを結合

### csv保存

# 関数作成

## ベースurlからurlリスト(urls)作成

In [12]:
def get_urls(base_url = "", url = ""):
    '''
    base_url: url検索を開始するベースurl
    urls: urlリストを格納するリスト
    '''
    urls = []
    
    if base_url == "":
        base_url = "http://suumo.jp"
    print("1\t", url)

    # 格納していく
    urls.append(url)
    flg = True
    i = 1
    while flg:
        #if i == 300:
        #    break
        i += 1
        request = requests.get(url)
        c = request.content
        soup = BeautifulSoup(c)
        # 次ページのリンクが入った<p>を取得
        next_page_ls = soup.find_all("p",{"class":"pagination-parts"})
        for next_page in next_page_ls:
            flg = True
            next_page_link = next_page.a
            # 次のページが見つからなかったら終了
            if "次へ" not in next_page_link:
                flg = False
            # 次のページが見つかれば、リンク取得
            if flg:
                url = base_url + next_page_link["href"]
                print(i, "\t", url)
                urls.append(url)
                # プログラムを最低1秒間停止する（スクレイピングマナー）
                time.sleep(4)
                
    print("終了：ページ数 = {}".format(str(i-1)))
    return urls

## 対象urlから物件情報を取得

In [13]:
def get_cassetteitems_info(urls):
    '''
    urls: 物件検索結果のurlリスト
    '''

    # データ（属性）を格納する変数を作成
    titles = [] #建物名（違う場合もあるが念のため）
    ltypes = [] #レンタルタイプ（賃貸アパート、賃貸マンション、など）
    addresses = [] #住所
    location0s = [] #立地1つ目（最寄駅/徒歩~分）
    location1s = [] #立地2つ目（最寄駅/徒歩~分）
    location2s = [] #立地3つ目（最寄駅/徒歩~分）
    ages = [] #築年数
    heights = [] #建物高さ
    floors = [] #階
    rents = [] #賃料
    admins = [] #管理費
    others = [] #敷/礼/保証/敷引,償却
    floor_plans = [] #間取り
    areas = [] #専有面積

    rooms = 0 # 物件数
    i = 0 # ページ数
    for url in tqdm(urls):
        i += 1 #ページ数カウント
        # 物件情報が入っている箇所を取得
        result = requests.get(url)
        c = result.content
        soup = BeautifulSoup(c)
        summary = soup.find("div",{"id":"js-bukkenList"})

        # 物件（建物）ごとの情報を取得
        cassetteitems = summary.find_all("div",{"class":"cassetteitem"})

        rooms += len(cassetteitems) #物件数カウント
        print("{}/{}\t物件（建物）数：{}".format(str(i), str(len(urls)), len(cassetteitems)))
        # 各物件で情報を取得
        for cassetteitem in cassetteitems:
            # 部屋数を取得
            tbodies = cassetteitem.find_all("tbody")

            # 建物情報を取得：レンタルタイプ、建物名、住所、立地（3つまで）、築年数、建物高さ
            # - レンタルタイプ
            ltype = cassetteitem.find("span",{"class":"ui-pct ui-pct--util1"}).find(text=True)
            # - 建物名
            title = cassetteitem.find("div",{"class":"cassetteitem_content-title"}).find(text=True)
            # - 住所
            address = cassetteitem.find("li",{"class":"cassetteitem_detail-col1"}).find(text=True)
            # - 立地（3つまで）
            locations = cassetteitem.find_all("div",{"class":"cassetteitem_detail-text"})
            l = len(locations)
            if l == 1:
                location0 = str(locations[0].find(text=True))
            if l == 2:
                location0 = str(locations[0].find(text=True))
                location1 = str(locations[1].find(text=True))
            else:
                location0 = str(locations[0].find(text=True))
                location1 = str(locations[1].find(text=True))
                location2 = str(locations[2].find(text=True))
            # 築年数、建物高さ
            ah = cassetteitem.find("li",{"class":"cassetteitem_detail-col3"})
            ah = ah.find_all("div")
            age = str(ah[0].find(text=True))
            height = str(ah[1].find(text=True))

            # 部屋情報を取得：階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積
            for tbody in tbodies:
                cassetteitem_others = tbody.find_all("td")
                # 階
                floor = str(cassetteitem_others[2].find(text=True))
                # 賃料
                rent = str(cassetteitem_others[3].find("span",{"class":"cassetteitem_other-emphasis ui-text--bold"}).find(text=True))
                # 管理費
                admin = str(cassetteitem_others[4].find(text=True))
                # 敷/礼/保証/敷引,償却
                other = str(cassetteitem_others[5].find(text=True))
                # 間取り
                floor_plan = str(cassetteitem_others[6].find(text=True))
                # 専有面積
                area = str(cassetteitem_others[7].find(text=True))

                # 部屋ごとにデータ（属性）を格納
                titles.append(title) #建物名
                ltypes.append(ltype) #レンタルタイプ
                addresses.append(address) #住所
                location0s.append(location0) #立地1つ目
                location1s.append(location1) #立地2つ目
                location2s.append(location2) #立地3つ目
                ages.append(age) #築年数
                heights.append(height) #建物高さ
                floors.append(floor) #階
                rents.append(rent) #賃料
                admins.append(admin) #管理費
                others.append(other) #敷/礼/保証/敷引,償却
                floor_plans.append(floor_plan) #間取り
                areas.append(area) #専有面積            

        # プログラムを最低1秒間停止する（スクレイピングマナー）
        time.sleep(4)

    print("ページ数：{}\t物件数：{}\t部屋数：{}".format(str(i), str(rooms), str(len(ltypes))))

    # データ格納した変数をSeries化
    titles = pd.Series(titles) #建物名
    ltypes = pd.Series(ltypes) #レンタルタイプ
    addresses = pd.Series(addresses) #住所
    location0s = pd.Series(location0s) #立地1つ目
    location1s = pd.Series(location1s) #立地2つ目
    location2s = pd.Series(location2s) #立地3つ目
    ages = pd.Series(ages) #築年数
    heights = pd.Series(heights) #建物高さ
    floors = pd.Series(floors) #階
    rents = pd.Series(rents) #賃料
    admins = pd.Series(admins) #管理費
    others = pd.Series(others) #敷/礼/保証/敷引,償却
    floor_plans = pd.Series(floor_plans) #間取り
    areas = pd.Series(areas) #専有面積

    # 各SeriesをDataFrame化
    df = pd.concat([titles, ltypes, addresses, location0s, location1s, location2s, ages, heights, floors, rents, admins, others, floor_plans, areas], axis = 1)

    # 列名を追加
    df.columns = ["建物名", "レンタルタイプ", "住所", "立地1", "立地2", "立地3", "築年数", "建物高さ", "階", "賃料", "管理費", "敷/礼/保証/敷引,償却", "間取り", "専有面積"]

    return df

## 立地を分割してpd.DataFrameで返す関数

In [14]:
def get_df_locations(arr_loc, stem = ""):
    '''
    arr_loc: 立地情報が入っている配列（例１）日暮里・舎人ライナー/高野駅 歩9分
    　　　　　　　　　　　　　　　　  （例２）東京メトロ千代田線/北千住駅 バス11分 (バス停)本木新道 歩1分
    => df_locationを作成して返す。
    df_location: 立地情報を格納するpd.DataFrame, columns={"路線","駅","バス","徒歩","確認用"}                      

    '''
    columns_ = [stem+"_路線",stem+"_駅",stem+"_バス",stem+"_徒歩",stem+"_確認用"]
    df_location = pd.DataFrame(columns=columns_)
    for elem in tqdm(arr_loc):
        # 情報取得
        if elem is None or elem == "" or elem == "None":
            line = "-"
            station = "-"
            bus = 0
            walk = 0
            elem = "None"
        else:
            arr1 = str(elem).split("/")
            line = str(arr1[0])
            bus = 0
            if len(arr1) == 1:
                arr1.append("")
            if " バス" in arr1[1]:
                bus_ = re.search("バス[0-9]+分", arr1[1]).group()
                bus = re.search("[0-9]+", bus_).group()
            walk = 0
            if " 歩" in arr1[1]:
                walk_ = re.search("歩[0-9]+分", arr1[1]).group()
                walk = re.search("[0-9]+", walk_).group()
            arr2 = arr1[1].split(" ")
            station = str(arr2[0])
        # dfへ格納
        df_ = pd.DataFrame([[line, station, bus, walk, elem]], columns=columns_)
        df_location = df_location.append(df_, ignore_index=True)
        df_location.columns = columns_
    return df_location

# 23区分のデータを一括取得

## 23区の1ページ目のurlリスト作成

In [5]:
all_first_urls = ["http://suumo.jp/chintai/tokyo/sc_chiyoda/",
                  "http://suumo.jp/chintai/tokyo/sc_chuo/",
                  "http://suumo.jp/chintai/tokyo/sc_minato/",
                  "http://suumo.jp/chintai/tokyo/sc_shinjuku/",
                  "http://suumo.jp/chintai/tokyo/sc_bunkyo/",
                  "http://suumo.jp/chintai/tokyo/sc_taito/",
                  "http://suumo.jp/chintai/tokyo/sc_sumida/",
                  "http://suumo.jp/chintai/tokyo/sc_koto/",
                  "http://suumo.jp/chintai/tokyo/sc_shinagawa/",
                  "http://suumo.jp/chintai/tokyo/sc_meguro/",
                  "http://suumo.jp/chintai/tokyo/sc_ota/",
                  "http://suumo.jp/chintai/tokyo/sc_setagaya/",
                  "http://suumo.jp/chintai/tokyo/sc_shibuya/",
                  "http://suumo.jp/chintai/tokyo/sc_nakano/",
                  "http://suumo.jp/chintai/tokyo/sc_suginami/",
                  "http://suumo.jp/chintai/tokyo/sc_toshima/",
                  "http://suumo.jp/chintai/tokyo/sc_kita/",
                  "http://suumo.jp/chintai/tokyo/sc_arakawa/",
                  "http://suumo.jp/chintai/tokyo/sc_itabashi/",
                  "http://suumo.jp/chintai/tokyo/sc_nerima/",
                  "http://suumo.jp/chintai/tokyo/sc_adachi/",
                  "http://suumo.jp/chintai/tokyo/sc_katsushika/",
                  "http://suumo.jp/chintai/tokyo/sc_edogawa/"]

In [6]:
ku_list = ["chiyoda", "chuo", "minato", "shinjuku", "bunkyo", "taito", "sumida", "koto", "shinagawa",
          "meguro", "ota", "setagaya", "shibuya", "nakano", "suginami", "toshima", "kita", "arakawa",
          "itabashi", "nerima", "adachi", "katsushika", "edogawa"]

## 23区のurlリストとdfを一括作成

In [7]:
for i in range(11,23):
    print("i={}\tku={}\turl={}".format(str(i),ku_list[i], all_first_urls[i]))

i=11	ku=setagaya	url=http://suumo.jp/chintai/tokyo/sc_setagaya/
i=12	ku=shibuya	url=http://suumo.jp/chintai/tokyo/sc_shibuya/
i=13	ku=nakano	url=http://suumo.jp/chintai/tokyo/sc_nakano/
i=14	ku=suginami	url=http://suumo.jp/chintai/tokyo/sc_suginami/
i=15	ku=toshima	url=http://suumo.jp/chintai/tokyo/sc_toshima/
i=16	ku=kita	url=http://suumo.jp/chintai/tokyo/sc_kita/
i=17	ku=arakawa	url=http://suumo.jp/chintai/tokyo/sc_arakawa/
i=18	ku= itabashi	url=http://suumo.jp/chintai/tokyo/sc_itabashi/
i=19	ku=nerima	url=http://suumo.jp/chintai/tokyo/sc_nerima/
i=20	ku=adachi	url=http://suumo.jp/chintai/tokyo/sc_adachi/
i=21	ku=katsushika	url=http://suumo.jp/chintai/tokyo/sc_katsushika/
i=22	ku=edogawa	url=http://suumo.jp/chintai/tokyo/sc_edogawa/


In [8]:
today = "20171119"
base_url = "http://suumo.jp"
#j = 0
#for first_url in all_first_urls:
for j in range(0, 23):
    first_url = all_first_urls[j]
    print("*"*100)
    print("{}区/23区\t{}".format(str(j+1), str(ku_list[j])))
    print("*"*100)
    # urlリストを作成
    urls = get_urls(base_url, first_url)
    # urlリストをcsvファイル出力
    urls_df = pd.DataFrame({"url":urls})
    urls_filename = "../output/urls/urls_" + ku_list[j] + "_" + today + ".csv"
    urls_df.to_csv(urls_filename, index=False, header=False)

    # 物件データを取得
    df = get_cassetteitems_info(urls)
    # 物件データをcsvファイル出力
    df_filename = "../output/info/info_" + ku_list[j] + "_" + today + ".csv"
    df.to_csv(df_filename, sep = ",", encoding = "utf-8")    
    
    # サイクルを回す
    #j += 1
    

****************************************************************************************************
12区/23区	setagaya
****************************************************************************************************
1	 http://suumo.jp/chintai/tokyo/sc_setagaya/


/Users/kimiaki/Documents/virtualenv/py3.5/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


2 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz12.html
3 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz13.html
4 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz14.html
5 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz15.html
6 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz16.html
7 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz17.html
8 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz18.html
9 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz19.html
10 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz110.html
11 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz111.html
12 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz112.html
13 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz113.html
14 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz114.html
15 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz115.html
16 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz116.html
17 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz117.html
18 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz118.html
19 	 http://s

140 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1140.html
141 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1141.html
142 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1142.html
143 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1143.html
144 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1144.html
145 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1145.html
146 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1146.html
147 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1147.html
148 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1148.html
149 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1149.html
150 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1150.html
151 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1151.html
152 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1152.html
153 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1153.html
154 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1154.html
155 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1155.html
156 	 http://suumo.jp/ch

275 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1275.html
276 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1276.html
277 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1277.html
278 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1278.html
279 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1279.html
280 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1280.html
281 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1281.html
282 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1282.html
283 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1283.html
284 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1284.html
285 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1285.html
286 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1286.html
287 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1287.html
288 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1288.html
289 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1289.html
290 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1290.html
291 	 http://suumo.jp/ch

...

137 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1137.html
138 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1138.html
139 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1139.html
140 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1140.html
141 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1141.html
142 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1142.html
143 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1143.html
144 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1144.html
145 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1145.html
146 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1146.html
147 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1147.html
148 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1148.html
149 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1149.html
150 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1150.html
151 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1151.html
152 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1152.html
153 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1153.ht

180/219	物件（建物）数：30
181/219	物件（建物）数：30
182/219	物件（建物）数：30
183/219	物件（建物）数：30
184/219	物件（建物）数：30
185/219	物件（建物）数：30
186/219	物件（建物）数：30
187/219	物件（建物）数：30
188/219	物件（建物）数：30
189/219	物件（建物）数：30
190/219	物件（建物）数：30
191/219	物件（建物）数：30
192/219	物件（建物）数：30
193/219	物件（建物）数：30
194/219	物件（建物）数：30
195/219	物件（建物）数：30
196/219	物件（建物）数：30
197/219	物件（建物）数：30
198/219	物件（建物）数：30
199/219	物件（建物）数：30
200/219	物件（建物）数：30
201/219	物件（建物）数：30
202/219	物件（建物）数：30
203/219	物件（建物）数：30
204/219	物件（建物）数：30
205/219	物件（建物）数：30
206/219	物件（建物）数：30
207/219	物件（建物）数：30
208/219	物件（建物）数：30
209/219	物件（建物）数：30
210/219	物件（建物）数：30
211/219	物件（建物）数：30
212/219	物件（建物）数：30
213/219	物件（建物）数：30
214/219	物件（建物）数：30
215/219	物件（建物）数：30
216/219	物件（建物）数：30
217/219	物件（建物）数：30
218/219	物件（建物）数：30
219/219	物件（建物）数：11
ページ数：219	物件数：6551	部屋数：11929
****************************************************************************************************
22区/23区	katsushika
********************************************************************************************

119 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1119.html
120 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1120.html
121 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1121.html
122 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1122.html
123 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1123.html
124 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1124.html
125 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1125.html
126 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1126.html
127 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1127.html
128 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1128.html
129 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1129.html
130 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1130.html
131 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1131.html
132 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1132.html
133 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1133.html
134 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz11

69 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz169.html
70 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz170.html
71 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz171.html
72 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz172.html
73 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz173.html
74 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz174.html
75 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz175.html
76 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz176.html
77 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz177.html
78 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz178.html
79 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz179.html
80 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz180.html
81 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz181.html
82 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz182.html
83 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz183.html
84 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz184.html
85 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz185.html
86 	 http://su

207 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1207.html
208 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1208.html
209 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1209.html
210 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1210.html
211 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1211.html
212 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1212.html
213 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1213.html
214 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1214.html
215 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1215.html
216 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1216.html
217 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1217.html
218 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1218.html
219 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1219.html
220 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1220.html
221 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1221.html
222 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1222.html
223 	 http://suumo.jp/chintai/tokyo/sc_e

- => 終わってから、「ページアクセス→情報取得→次ページあればサイクル回す」にすれば時間半分で済んだなーと反省。
- 参考サイトの流れを意識しすぎたことが原因で、自分のコードに落とした場合を考慮し、修正が必要でした。ちゃんちゃん。

# データ前処理

## データ読み込み

In [3]:
# 空のDataFrame作成
df_adachi = pd.read_csv("../output/info/info_adachi_20171119.csv", encoding="utf-8")
df_all = pd.DataFrame(columns=df_adachi.columns)

In [4]:
df_all

,Unnamed: 0,建物名,レンタルタイプ,住所,立地1,立地2,立地3,築年数,建物高さ,階,賃料,管理費,"敷/礼/保証/敷引,償却",間取り,専有面積


In [8]:
# 23区分を読み込みdf_allへ結合
path = "../output/info/"
info_files = os.listdir(path)
#print(info_files)
for info_file in info_files:
    if ".csv" not in info_file:
        continue
    file_path = path + info_file
    df_all_ = pd.read_csv(file_path, encoding="utf-8")
    df_all = df_all.append(df_all_, ignore_index=True)
df_all = df_all.reset_index(drop=True)
print(df_all.shape)
df_all.head(10)

(623994, 15)


,Unnamed: 0,建物名,レンタルタイプ,住所,立地1,立地2,立地3,築年数,建物高さ,階,賃料,管理費,"敷/礼/保証/敷引,償却",間取り,専有面積
0,0,東京メトロ千代田線 綾瀬駅 2階建 築33年,賃貸アパート,東京都足立区西綾瀬４,東京メトロ千代田線/綾瀬駅 歩11分,つくばエクスプレス/青井駅 歩12分,東武伊勢崎線/小菅駅 歩15分,築33年,2階建,2階,2.5万円,1000円,2.5万円/2.5万円/-/-,2K,24m
1,1,小泉荘,賃貸アパート,東京都足立区小台２,日暮里・舎人ライナー/足立小台駅 歩17分,ＪＲ高崎線/尾久駅 歩27分,日暮里・舎人ライナー/熊野前駅 歩21分,築48年,2階建,1階,2.6万円,-,2.6万円/2.6万円/-/-,ワンルーム,11.57m
2,2,東京メトロ南北線 王子神谷駅 2階建 築43年,賃貸アパート,東京都足立区新田１,東京メトロ南北線/王子神谷駅 歩15分,None,None,築43年,2階建,1階,2.5万円,1500円,2.5万円/2.5万円/-/-,1K,12m
3,3,文月荘,賃貸アパート,東京都足立区新田１,東京メトロ南北線/王子神谷駅 歩17分,None,None,築53年,2階建,2階,2.5万円,1500円,2.5万円/2.5万円/-/-,ワンルーム,10.01m
4,4,文月荘,賃貸アパート,東京都足立区新田１,東京メトロ南北線/王子神谷駅 歩17分,None,None,築53年,2階建,1階,2.5万円,1500円,2.5万円/2.5万円/-/-,ワンルーム,10.01m
5,5,豊和荘,賃貸アパート,東京都足立区梅島２,東武伊勢崎線/梅島駅 歩9分,東武伊勢崎線/西新井駅 歩13分,東武大師線/大師前駅 歩23分,築58年,2階建,1階,2.8万円,-,2.8万円/-/-/-,ワンルーム,10m
6,6,豊和荘,賃貸アパート,東京都足立区梅島２,東武伊勢崎線/梅島駅 歩9分,東武伊勢崎線/西新井駅 歩13分,東武大師線/大師前駅 歩23分,築58年,2階建,1階,2.8万円,-,2.8万円/-/-/-,ワンルーム,10m
7,7,柳荘,賃貸アパート,東京都足立区千住曙町,東京メトロ千代田線/北千住駅 歩15分,京成本線/京成関屋駅 歩1分,東武伊勢崎線/牛田駅 歩1分,築47年,2階建,2階,2.8万円,-,2.8万円/2.8万円/-/-,ワンルーム,10m
8,8,柳川荘,賃貸アパート,東京都足立区千住曙町,東京メトロ千代田線/北千住駅 歩15分,東武伊勢崎線/牛田駅 歩1分,京成本線/京成関屋駅 歩1分,築47年,2階建,1階,2.8万円,-,2.8万円/2.8万円/-/-,ワンルーム,10m
9,9,柳川荘,賃貸アパート,東京都足立区千住曙町,東京メトロ千代田線/北千住駅 歩15分,東武伊勢崎線/牛田駅 歩1分,京成本線/京成関屋駅 歩1分,築47年,2階建,2階,2.8万円,-,2.8万円/2.8万円/-/-,ワンルーム,10m


## 不要(賃料がNA)行を削除

In [9]:
print(df_all.shape)
df_all = df_all.dropna(subset=["賃料"])
print(df_all.shape)

(623994, 15)
(623994, 15)


## 列を分割

In [19]:
## 住所
# 住所を都道府県、市区町村、町域以下、に分割（東京都/〇〇区/それ以下）
df_address1 = df_all["住所"].str.split("都", expand=True)
df_address1.columns = ["都道府県", "それ以下"]
df_address2 = df_address1["それ以下"].str.split("区", expand=True)
df_address2.columns = ["市区町村", "町域以下"]
df_address = pd.concat([df_address1["都道府県"], df_address2], axis=1)
# データ確認
print(df_address.shape)
df_address.head(10)

 23%|██▎       | 1164/5000 [00:25<01:22, 46.34it/s]

(623994, 3)


,都道府県,市区町村,町域以下
0,東京,足立,西綾瀬４
1,東京,足立,小台２
2,東京,足立,新田１
3,東京,足立,新田１
4,東京,足立,新田１
5,東京,足立,梅島２
6,東京,足立,梅島２
7,東京,足立,千住曙町
8,東京,足立,千住曙町
9,東京,足立,千住曙町


In [22]:
## 立地
# 立地を項目に分割
df_location0s = []
df_location1s = []
df_location2s = []
# データ量多いので分割処理
print("*"*20, " 立地1 ", "*"*20)
for locs in itertools.zip_longest(*[iter(df_all["立地1"])]*5000):
    df_tmp = get_df_locations(arr_loc=locs, stem="立地1")
    df_location0s.append(df_tmp)
print("*"*20, " 立地2 ", "*"*20)    
for locs in itertools.zip_longest(*[iter(df_all["立地2"])]*5000):
    df_tmp = get_df_locations(arr_loc=locs, stem="立地2")
    df_location1s.append(df_tmp)
print("*"*20, " 立地3 ", "*"*20)    
for locs in itertools.zip_longest(*[iter(df_all["立地3"])]*5000):
    df_tmp = get_df_locations(arr_loc=locs, stem="立地3")
    df_location2s.append(df_tmp)


  1%|          | 31/5000 [00:00<00:16, 300.82it/s]

********************  立地1  ********************



  2%|▏         | 90/5000 [00:00<00:10, 448.31it/s]

********************  立地2  ********************


  1%|          | 44/5000 [00:00<00:11, 434.30it/s]

********************  立地3  ********************


100%|██████████| 5000/5000 [00:15<00:00, 313.47it/s]


'\nfor i in range(1, len(df_all["立地1"] + 1))\ndf_location0 = get_df_locations(arr_loc=df_all["立地1"], stem="立地1")\ndf_location1 = get_df_locations(arr_loc=df_all["立地2"], stem="立地2")\ndf_location2 = get_df_locations(arr_loc=df_all["立地3"], stem="立地3")\ndf_locations = pd.concat([df_location0.ix[:, 0:4], df_location1.ix[:, 0:4], df_location2.ix[:, 0:4]], axis=1)\n# データ確認\ndf_locations.shape\ndf_locations.head(10)\n'

In [107]:
# df_locationXsを縦に結合
df_location0_ = pd.concat(df_location0s)
df_location1_ = pd.concat(df_location1s)
df_location2_ = pd.concat(df_location2s)
#print(df_location0_.shape)
#print(df_location1_.shape)
#print(df_location2_.shape)
# 不要な行と列を除き、横に結合してdf_locationsを作成
l = len(df_all["立地1"])
df_locations = pd.concat([df_location0_.iloc[:l, :4], df_location1_.iloc[:l, :4], df_location2_.iloc[:l, :4]], axis=1)
#df_location0 = df_location0_.iloc[:l, :4]
#df_location1 = df_location1_.iloc[:l, :4]
#df_location2 = df_location2_.iloc[:l, :4]
#print(df_location0.shape)
# index振り直し
df_locations = df_locations.reset_index(drop=True)
# データ確認
print(df_locations.shape)
df_locations.head(10)

(623994, 12)


,立地1_路線,立地1_駅,立地1_バス,立地1_徒歩,立地2_路線,立地2_駅,立地2_バス,立地2_徒歩,立地3_路線,立地3_駅,立地3_バス,立地3_徒歩
0,東京メトロ千代田線,綾瀬駅,0,11,つくばエクスプレス,青井駅,0,12,東武伊勢崎線,小菅駅,0,15
1,日暮里・舎人ライナー,足立小台駅,0,17,ＪＲ高崎線,尾久駅,0,27,日暮里・舎人ライナー,熊野前駅,0,21
2,東京メトロ南北線,王子神谷駅,0,15,-,-,0,0,-,-,0,0
3,東京メトロ南北線,王子神谷駅,0,17,-,-,0,0,-,-,0,0
4,東京メトロ南北線,王子神谷駅,0,17,-,-,0,0,-,-,0,0
5,東武伊勢崎線,梅島駅,0,9,東武伊勢崎線,西新井駅,0,13,東武大師線,大師前駅,0,23
6,東武伊勢崎線,梅島駅,0,9,東武伊勢崎線,西新井駅,0,13,東武大師線,大師前駅,0,23
7,東京メトロ千代田線,北千住駅,0,15,京成本線,京成関屋駅,0,1,東武伊勢崎線,牛田駅,0,1
8,東京メトロ千代田線,北千住駅,0,15,東武伊勢崎線,牛田駅,0,1,京成本線,京成関屋駅,0,1
9,東京メトロ千代田線,北千住駅,0,15,東武伊勢崎線,牛田駅,0,1,京成本線,京成関屋駅,0,1


↑ここでindex振り直ししないと、後でpd.concatでdataframe結合できなくなるので、忘れないように！

In [62]:
## 築年数
# 築年数から数値のみ取得
df_age = df_all["築年数"].str.replace("新築", "築0年")
df_age = df_age.str.extract("([0-9]+)").astype(int)
df_age.columns = ["築年数"]
# データ確認
print(df_age.shape)
Counter(df_age)

/Users/kimiaki/Documents/virtualenv/py3.5/lib/python3.5/site-packages/ipykernel_launcher.py:4: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  after removing the cwd from sys.path.


(623994,)


Counter({0: 82407,
         1: 7032,
         2: 17733,
         3: 15846,
         4: 11523,
         5: 10614,
         6: 9393,
         7: 9147,
         8: 9642,
         9: 12816,
         10: 16866,
         11: 20328,
         12: 19422,
         13: 17202,
         14: 16671,
         15: 15198,
         16: 12294,
         17: 9861,
         18: 9504,
         19: 8757,
         20: 9270,
         21: 9525,
         22: 8316,
         23: 9471,
         24: 11826,
         25: 13866,
         26: 15735,
         27: 19119,
         28: 20958,
         29: 21078,
         30: 19128,
         31: 16128,
         32: 12630,
         33: 11979,
         34: 8553,
         35: 6900,
         36: 5451,
         37: 5130,
         38: 7791,
         39: 5934,
         40: 5616,
         41: 4956,
         42: 3876,
         43: 4527,
         44: 4008,
         45: 5448,
         46: 3936,
         47: 3492,
         48: 4035,
         49: 2271,
         50: 1761,
         51: 1503,

In [63]:
## 建物高さ
# 平屋を1階建に変換
df_height1 = df_all["建物高さ"].str.replace("平屋", "1階建")
# 地下と地上に分割
df_height1 = pd.DataFrame(["地上" + x if "地上" not in x else x for x in df_height1], columns=["建物高さ"])
df_height2 = df_height1["建物高さ"].str.split("地上", expand=True)
df_height2.columns = ["地下", "地上"]
df_height2["地下"] = ["地下0" if x == "" else x for x in df_height2["地下"]]
# 地下と地上の数値のみ取得
df_height2["地下"] = df_height2["地下"].str.extract("([0-9]+)").astype(int)
df_height2["地上"] = df_height2["地上"].str.extract("([0-9]+)").astype(int)
# 地下はマイナス値にする
df_height2["地下"] = df_height2["地下"] * -1
# データ確認
print(df_height2.shape)
print(Counter(df_height2["地下"]))
print(Counter(df_height2["地上"]))
df_height2.head(10)

/Users/kimiaki/Documents/virtualenv/py3.5/lib/python3.5/site-packages/ipykernel_launcher.py:10: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  # Remove the CWD from sys.path while we load stuff.
/Users/kimiaki/Documents/virtualenv/py3.5/lib/python3.5/site-packages/ipykernel_launcher.py:11: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  # This is added back by InteractiveShellApp.init_path()


(623994, 2)
Counter({0: 588933, -1: 27543, -2: 6078, -3: 948, -4: 303, -6: 66, -7: 45, -8: 39, -5: 36, -9: 3})
Counter({2: 163875, 3: 125685, 4: 69147, 5: 57252, 7: 26460, 6: 26418, 10: 24528, 8: 23076, 11: 18945, 9: 17853, 14: 17214, 12: 16524, 13: 14352, 15: 8493, 24: 1269, 30: 801, 1: 783, 20: 777, 40: 729, 16: 714, 60: 666, 22: 576, 19: 570, 17: 480, 38: 453, 18: 381, 39: 372, 32: 366, 43: 366, 25: 354, 41: 351, 33: 285, 23: 282, 27: 264, 48: 252, 21: 231, 36: 225, 42: 225, 58: 225, 52: 210, 29: 198, 31: 195, 28: 171, 49: 159, 44: 150, 47: 147, 56: 132, 37: 126, 34: 111, 45: 108, 26: 99, 53: 99, 46: 84, 35: 81, 55: 75, 50: 21, 54: 9})


,地下,地上
0,0,2
1,0,2
2,0,2
3,0,2
4,0,2
5,0,2
6,0,2
7,0,2
8,0,2
9,0,2


In [64]:
## 階
# 「-」を「1階」に変換
df_floor = pd.DataFrame(df_all["階"], columns=["階"])
df_floor = pd.DataFrame(["1階" if x == "-" else x for x in df_floor["階"]], columns=["階"])
#print(Counter(df_floor1["階"]))
# 「○-△階」の処理
df_floor = df_floor["階"].str.split("-", expand=True)
df_floor.columns = ["階", "その他"]
# Bはマイナス値とし、階数を取得
df_floor = pd.DataFrame(df_floor["階"].str.replace("階", ""), columns=["階"])
df_floor = pd.DataFrame(df_floor["階"].str.replace("B", "-"), columns=["階"])
df_floor["階"] = pd.to_numeric(df_floor["階"])
# データ確認
print(df_floor.shape)
Counter(df_floor["階"])

(623994, 1)


Counter({-6: 3,
         -5: 6,
         -4: 3,
         -3: 21,
         -2: 51,
         -1: 678,
         1: 147456,
         2: 190377,
         3: 104367,
         4: 57261,
         5: 32544,
         6: 21252,
         7: 16602,
         8: 13218,
         9: 10317,
         10: 8142,
         11: 5823,
         12: 4296,
         13: 2766,
         14: 1965,
         15: 942,
         16: 540,
         17: 399,
         18: 480,
         19: 360,
         20: 429,
         21: 282,
         22: 324,
         23: 249,
         24: 348,
         25: 195,
         26: 159,
         27: 216,
         28: 192,
         29: 228,
         30: 162,
         31: 66,
         32: 144,
         33: 96,
         34: 111,
         35: 111,
         36: 69,
         37: 102,
         38: 75,
         39: 84,
         40: 51,
         41: 54,
         42: 30,
         43: 54,
         44: 48,
         45: 57,
         46: 18,
         47: 24,
         48: 9,
         49: 3,
         50: 21,
 

In [65]:
## 賃料
# 賃料の数値のみ取得し、10,000倍する。
df_rent = pd.DataFrame(round(pd.to_numeric(df_all["賃料"].str.replace("万円", "")) * 10000, 0), columns=["賃料"])
# データ確認
print(df_rent.shape)
df_rent.head(10)
#Counter(df_rent["賃料"])

(623994, 1)


,賃料
0,25000.0
1,26000.0
2,25000.0
3,25000.0
4,25000.0
5,28000.0
6,28000.0
7,28000.0
8,28000.0
9,28000.0


In [66]:
## 管理費
df_admin = pd.DataFrame(df_all["管理費"].str.replace("円", ""), columns=["管理費"])
df_admin = pd.DataFrame(pd.to_numeric(["0" if x == "-" else x for x in df_admin["管理費"]]) , columns=["管理費"])
# データ確認
print(df_admin.shape)
df_admin.head(10)
#Counter(df_admin["管理費"])

(623994, 1)


,管理費
0,1000
1,0
2,1500
3,1500
4,1500
5,0
6,0
7,0
8,0
9,0


In [67]:
## 敷/礼/保証/敷引,償却
# 「-」「実費」を0に変換
df_all["敷/礼/保証/敷引,償却"] = df_all["敷/礼/保証/敷引,償却"].str.replace("-", "0")
df_all["敷/礼/保証/敷引,償却"] = df_all["敷/礼/保証/敷引,償却"].str.replace("実費", "0")
df_others = pd.DataFrame(df_all["敷/礼/保証/敷引,償却"], columns=["敷/礼/保証/敷引,償却"])
# "万円"を削除
df_others = pd.DataFrame(df_others["敷/礼/保証/敷引,償却"].str.replace("万円", ""), columns=["敷/礼/保証/敷引,償却"])
# 数値取得し10,000倍
df_others = df_others["敷/礼/保証/敷引,償却"].str.split("/", expand=True).astype(float) * 10000
df_others.columns = ["敷金", "礼金", "保証金", "敷引,償却費"]
# データ確認
print(df_others.shape)
df_others.head(10)

(623994, 4)


,敷金,礼金,保証金,"敷引,償却費"
0,25000.0,25000.0,0.0,0.0
1,26000.0,26000.0,0.0,0.0
2,25000.0,25000.0,0.0,0.0
3,25000.0,25000.0,0.0,0.0
4,25000.0,25000.0,0.0,0.0
5,28000.0,0.0,0.0,0.0
6,28000.0,0.0,0.0,0.0
7,28000.0,28000.0,0.0,0.0
8,28000.0,28000.0,0.0,0.0
9,28000.0,28000.0,0.0,0.0


In [68]:
## 間取り
# ここは敢えてarrでの変換で処理する。
df_floor_plan = df_all["間取り"].str.replace("ワンルーム", "1")
# arr作成
l = len(df_floor_plan)
arr_rooms = np.zeros(l)
arr_d = np.zeros(l)
arr_k = np.zeros(l)
arr_l = np.zeros(l)
arr_s = np.zeros(l)
# 格納していく
for i in tqdm(range(0, l)):
    plan = str(df_floor_plan[i])
    # D有無
    if "D" in plan:
        arr_d[i] = 1
    plan = plan.replace("D", "")
    # K有無
    if "K" in plan:
        arr_k[i] = 1
    # L有無
    if "L" in plan:
        arr_l[i] = 1
    # S有無
    if "S" in plan:
        arr_s[i] = 1
    for t in ["D","K","L","S"]:
        plan = plan.replace(t, "")
    arr_rooms[i] = plan

# データ確認
print(Counter(arr_rooms))
print(Counter(arr_d))
print(Counter(arr_k))
print(Counter(arr_l))
print(Counter(arr_s))

# DataFrameへ変換
df_floor_plan = pd.DataFrame(columns=["部屋数","D有無","K有無","L有無","S有無"])
df_floor_plan["部屋数"] = arr_rooms
df_floor_plan["D有無"] = arr_d
df_floor_plan["K有無"] = arr_k
df_floor_plan["L有無"] = arr_l
df_floor_plan["S有無"] = arr_s
# データ確認
print(df_floor_plan.shape)
df_floor_plan.head(10)

100%|██████████| 623994/623994 [00:12<00:00, 48039.50it/s]


Counter({1.0: 478185, 2.0: 108354, 3.0: 32394, 4.0: 4041, 5.0: 810, 6.0: 132, 7.0: 51, 8.0: 18, 9.0: 6, 10.0: 3})
Counter({0.0: 396267, 1.0: 227727})
Counter({1.0: 511620, 0.0: 112374})
Counter({0.0: 493593, 1.0: 130401})
Counter({0.0: 614553, 1.0: 9441})
(623994, 5)


,部屋数,D有無,K有無,L有無,S有無
0,2.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0
9,1.0,0.0,0.0,0.0,0.0


In [69]:
## 専有面積
df_area = pd.DataFrame(round(pd.to_numeric(df_all["専有面積"].str.replace("m", "")), 2), columns=["専有面積"])
# データ確認
print(df_area.shape)
df_area.head(10)

(623994, 1)


,専有面積
0,24.00
1,11.57
2,12.00
3,10.01
4,10.01
5,10.00
6,10.00
7,10.00
8,10.00
9,10.00


In [70]:
## 月額費用・初期費用
df_cost = pd.DataFrame(columns=["月額費用", "初期費用"])
df_cost["月額費用"] = df_rent["賃料"] + df_admin["管理費"]
df_cost["初期費用"] = df_others["敷金"] + df_others["礼金"] + df_others["保証金"] + df_others["敷引,償却費"]
# データ確認
print(df_cost.shape)
df_cost.head(10)

(623994, 2)


,月額費用,初期費用
0,26000.0,50000.0
1,26000.0,52000.0
2,26500.0,50000.0
3,26500.0,50000.0
4,26500.0,50000.0
5,28000.0,28000.0
6,28000.0,28000.0
7,28000.0,56000.0
8,28000.0,56000.0
9,28000.0,56000.0


In [71]:
## 利用路線数（※df_locationsへ列を追加）
line_num = np.zeros(len(df_locations))
for id, row in tqdm(df_locations.iterrows()):
    if row[4] == "" and row[8] == "":
        line_num[id] = 1
    elif row[4] != "" and row[8] == "":
        line_num[id] = 2
    else:
        line_num[id] = 3
df_locations["利用路線数"] = line_num
# データ確認
print(df_locations.shape)
df_locations.head(10)

623994it [01:17, 8086.11it/s]

(623994, 13)


,立地1_路線,立地1_駅,立地1_バス,立地1_徒歩,立地2_路線,立地2_駅,立地2_バス,立地2_徒歩,立地3_路線,立地3_駅,立地3_バス,立地3_徒歩,利用路線数
0,東京メトロ千代田線,綾瀬駅,0,11,つくばエクスプレス,青井駅,0,12,東武伊勢崎線,小菅駅,0,15,3.0
1,日暮里・舎人ライナー,足立小台駅,0,17,ＪＲ高崎線,尾久駅,0,27,日暮里・舎人ライナー,熊野前駅,0,21,3.0
2,東京メトロ南北線,王子神谷駅,0,15,-,-,0,0,-,-,0,0,3.0
3,東京メトロ南北線,王子神谷駅,0,17,-,-,0,0,-,-,0,0,3.0
4,東京メトロ南北線,王子神谷駅,0,17,-,-,0,0,-,-,0,0,3.0
5,東武伊勢崎線,梅島駅,0,9,東武伊勢崎線,西新井駅,0,13,東武大師線,大師前駅,0,23,3.0
6,東武伊勢崎線,梅島駅,0,9,東武伊勢崎線,西新井駅,0,13,東武大師線,大師前駅,0,23,3.0
7,東京メトロ千代田線,北千住駅,0,15,京成本線,京成関屋駅,0,1,東武伊勢崎線,牛田駅,0,1,3.0
8,東京メトロ千代田線,北千住駅,0,15,東武伊勢崎線,牛田駅,0,1,京成本線,京成関屋駅,0,1,3.0
9,東京メトロ千代田線,北千住駅,0,15,東武伊勢崎線,牛田駅,0,1,京成本線,京成関屋駅,0,1,3.0


## 統合DataFrame作成

In [116]:
# 全てのデータフレームを結合
df_all_fix = pd.concat([df_all["レンタルタイプ"], df_address, df_locations, df_age, df_height2, df_floor["階"], df_rent["賃料"], df_admin["管理費"], df_others, df_floor_plan, df_area, df_cost], axis=1)
# データ確認
print(df_all_fix.shape)
df_all_fix

(623994, 34)


,レンタルタイプ,都道府県,市区町村,町域以下,立地1_路線,立地1_駅,立地1_バス,立地1_徒歩,立地2_路線,立地2_駅,...,保証金,"敷引,償却費",部屋数,D有無,K有無,L有無,S有無,専有面積,月額費用,初期費用
0,賃貸アパート,東京,足立,西綾瀬４,東京メトロ千代田線,綾瀬駅,0,11,つくばエクスプレス,青井駅,...,0.0,0.0,2.0,0.0,1.0,0.0,0.0,24.00,26000.0,50000.0
1,賃貸アパート,東京,足立,小台２,日暮里・舎人ライナー,足立小台駅,0,17,ＪＲ高崎線,尾久駅,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11.57,26000.0,52000.0
2,賃貸アパート,東京,足立,新田１,東京メトロ南北線,王子神谷駅,0,15,-,-,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,12.00,26500.0,50000.0
3,賃貸アパート,東京,足立,新田１,東京メトロ南北線,王子神谷駅,0,17,-,-,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.01,26500.0,50000.0
4,賃貸アパート,東京,足立,新田１,東京メトロ南北線,王子神谷駅,0,17,-,-,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.01,26500.0,50000.0
5,賃貸アパート,東京,足立,梅島２,東武伊勢崎線,梅島駅,0,9,東武伊勢崎線,西新井駅,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.00,28000.0,28000.0
6,賃貸アパート,東京,足立,梅島２,東武伊勢崎線,梅島駅,0,9,東武伊勢崎線,西新井駅,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.00,28000.0,28000.0
7,賃貸アパート,東京,足立,千住曙町,東京メトロ千代田線,北千住駅,0,15,京成本線,京成関屋駅,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.00,28000.0,56000.0
8,賃貸アパート,東京,足立,千住曙町,東京メトロ千代田線,北千住駅,0,15,東武伊勢崎線,牛田駅,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.00,28000.0,56000.0
9,賃貸アパート,東京,足立,千住曙町,東京メトロ千代田線,北千住駅,0,15,東武伊勢崎線,牛田駅,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.00,28000.0,56000.0


In [117]:
print(df_all_fix.shape)
df_all_fix.columns

(623994, 34)


Index(['レンタルタイプ', '都道府県', '市区町村', '町域以下', '立地1_路線', '立地1_駅', '立地1_バス',
       '立地1_徒歩', '立地2_路線', '立地2_駅', '立地2_バス', '立地2_徒歩', '立地3_路線', '立地3_駅',
       '立地3_バス', '立地3_徒歩', '築年数', '地下', '地上', '階', '賃料', '管理費', '敷金', '礼金',
       '保証金', '敷引,償却費', '部屋数', 'D有無', 'K有無', 'L有無', 'S有無', '専有面積', '月額費用',
       '初期費用'],
      dtype='object')

## csv保存

データ量が100MB以上になるため、二つに分割して保存する

In [21]:
df_all_fix = df_all_fix.ix[:, 1:]
df_all_fix_1 = df_all_fix.ix[:300000, :]
df_all_fix_2 = df_all_fix.ix[300001:, :]
#df_all_fix_1 = df_all_fix_1.reset_index(drop=True)
#df_all_fix_2 = df_all_fix_2.reset_index(drop=True)
print(df_all_fix_1.columns)
print(df_all_fix_1.shape)
print(df_all_fix_2.columns)
print(df_all_fix_2.shape)
df_all_fix_1.to_csv("../output/df_csv/df_all_fix_1.csv", sep = ",", encoding = "utf-8")
df_all_fix_2.to_csv("../output/df_csv/df_all_fix_2.csv", sep = ",", encoding = "utf-8")

/Users/kimiaki/Documents/virtualenv/py3.5/lib/python3.5/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  
/Users/kimiaki/Documents/virtualenv/py3.5/lib/python3.5/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


Index(['レンタルタイプ', '都道府県', '市区町村', '町域以下', '立地1_路線', '立地1_駅', '立地1_バス',
       '立地1_徒歩', '立地2_路線', '立地2_駅', '立地2_バス', '立地2_徒歩', '立地3_路線', '立地3_駅',
       '立地3_バス', '立地3_徒歩', '築年数', '地下', '地上', '階', '賃料', '管理費', '敷金', '礼金',
       '保証金', '敷引,償却費', '部屋数', 'D有無', 'K有無', 'L有無', 'S有無', '専有面積', '月額費用',
       '初期費用'],
      dtype='object')
(300001, 34)
Index(['レンタルタイプ', '都道府県', '市区町村', '町域以下', '立地1_路線', '立地1_駅', '立地1_バス',
       '立地1_徒歩', '立地2_路線', '立地2_駅', '立地2_バス', '立地2_徒歩', '立地3_路線', '立地3_駅',
       '立地3_バス', '立地3_徒歩', '築年数', '地下', '地上', '階', '賃料', '管理費', '敷金', '礼金',
       '保証金', '敷引,償却費', '部屋数', 'D有無', 'K有無', 'L有無', 'S有無', '専有面積', '月額費用',
       '初期費用'],
      dtype='object')
(323993, 34)
